In [ ]:

import os
import pdb
import argparse
import numpy as np
import tensorflow as tf
from scipy.stats import pearsonr
from tensorflow.keras import backend as K
from model.model_fn import build_compile_model_pred

Using TensorFlow backend.
from skimage.io import imshow
import matplotlib.pyplot as plt
#input_name = 'examples/samples_im.png'
input_name = 'examples/samples_nat_im.npy'
WEIGHT_DIR = 'weights.last.h5'
BATCH_SIZE = 64
# input
out_name, file_ext = os.path.splitext(input_name)

#
flag_numpy = 1 if file_ext=='.npy' else 0

# model
model = build_compile_model_pred(WEIGHT_DIR)
model.summary()

if flag_numpy:
    from skimage.transform import resize
    tmp_input = np.load(input_name)
    test_steps = 1

    NO_INPUT = tmp_input.shape[0]
    # resize
    test_input = np.zeros((NO_INPUT, 84, 84, 3))
    for im_id in range(tmp_input.shape[0]):
        test_input[im_id, :, :, :] = resize(tmp_input[im_id, :, :, :], (84, 84, 3), anti_aliasing=True)
        test_input[im_id, :, :, :] -= [103.939, 116.779, 123.68]
    pred = model.predict(test_input, steps=test_steps)
    np.save(out_name + '_pred.npy', pred)

else:
    from skimage.io import imread
    from skimage.transform import resize
    img = imread(input_name)

    # resize image to 84x84
    img = resize(img, (84, 84, 3), anti_aliasing=True)
    img -= [103.939, 116.779, 123.68]
    pred = model.predict(np.expand_dims(img, axis=0), steps=1)
    print(pred[0][0])

    NO_INPUT = 1
WARNING:tensorflow:From /mnt/gs/home/uranc/anaconda3/envs/tfGPU/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
loading model
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
=================================================================
input_1 (InputLayer)         [(None, 84, 84, 3)]       0
_________________________________________________________________
model (Model)                (None, 21, 21, 256)       1735488
_________________________________________________________________
squeeze_1 (Conv2D)           (None, 10, 10, 32)        73760
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 10, 10, 32)        0
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 32)        0
_________________________________________________________________
squeeze_2 (Conv2D)           (None, 4, 4, 16)          4624
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 16)          0
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 16)          0
_________________________________________________________________
conv_out (Conv2D)            (None, 1, 1, 1)           257
_________________________________________________________________
flatten (Flatten)            (None, 1)                 0
=================================================================
Total params: 1,814,129
Trainable params: 78,641
Non-trainable params: 1,735,488
_________________________________________________________________
plt.figure(figsize=(20,10))
for ii in range(NO_INPUT):
    plt.subplot(1,6,ii+1)
    plt.imshow(test_input[ii,:,:,:] + [103.939, 116.779, 123.68])


pred
array([[0.22771412],
       [0.21979913],
       [0.22885998],
       [0.22246471],
       [0.21555173],
       [0.22140911]], dtype=float32)